This is a bonus notebook that shows how we can implement convolution simply (if somewhat inefficiently) using strided memory to create "virtual" arrays against which to apply a convolution kernel.

In [ ]:
from scipy.signal import convolve2d

In [ ]:
data = np.arange(25).reshape(5, 5)
data

In [ ]:
kernel = np.eye(3)
kernel

In [ ]:
convolve2d(data, kernel, mode='valid')

In [ ]:
import numpy as np
from matplotlib.pyplot import imshow
from numpy.lib.stride_tricks import as_strided

data = np.arange(30).reshape(6, 5)
kernel = np.eye(3)


def extrude_for_convolution(data, kernel):
    # Indices of the top left corner of the last window.
    last_row = data.shape[0] - (kernel.shape[0])
    last_col = data.shape[1] - (kernel.shape[1])

    num_windows = last_row * data.shape[1] + last_col + 1

    shape = (num_windows,) + kernel.shape
    strides = (data.strides[1],) + data.strides

    # This is where the magic happens.
    windows = as_strided(data, shape=shape, strides=strides)
    
    # 
    valid = np.arange(num_windows) % data.shape[1] <= last_col

    return windows, valid


def convolve_2d(data, kernel):
    windows, valid = extrude_for_convolution(data, kernel)

    # A 1D array containing the results of multiplying each window by the
    # convolution kernel.
    multiplication_results = np.tensordot(windows, kernel)[valid]

    return multiplication_results.reshape(data.shape[0] - kernel.shape[0] + 1,
                                          data.shape[1] - kernel.shape[1] + 1)

In [ ]:
from skimage.data import horse
image = horse()
image.shape

In [ ]:
imshow(image)

In [ ]:
from scipy.signal import gaussian
kernel = gaussian(3, 1) + gaussian(3, 1)[:, np.newaxis]
kernel

In [ ]:
blurred = convolve_2d(image, kernel)
imshow(blurred)